In [21]:
import keras.utils
from sklearn.preprocessing import LabelEncoder
from keras.models import Sequential
from keras.layers import LSTM, Embedding, Dense, Conv1D, GlobalMaxPooling1D
from keras.preprocessing.text import Tokenizer
import tensorflow as tf
import pandas as pd
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from keras.utils import pad_sequences

In [22]:
# Load your dataset
data = pd.read_csv('TV.csv')

# Split the data into features (X) and labels (y)
X = data['Nhận xét đánh giá'].values
y = data['Cảm xúc'].values

# Encode the labels
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [25]:
# TF-IDF Vectorization
tfidf_vectorizer = TfidfVectorizer(max_features=5000)
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_val_tfidf = tfidf_vectorizer.transform(X_val)

# Apply SMOTE to balance the training data
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_tfidf, y_train)

# Tokenization and Padding
max_words = 5000
max_len = 1000
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(X_train)

X_train_seq = tokenizer.texts_to_sequences(X_train)
X_val_seq = tokenizer.texts_to_sequences(X_val)

X_train_seq = pad_sequences(X_train_seq, maxlen=max_len)
X_val_seq = pad_sequences(X_val_seq, maxlen=max_len)

# Build and compile your CNN model
embedding_dim = 120
num_filters = 128
kernel_size = 5
num_classes = len(label_encoder.classes_)

In [26]:

model = Sequential()
model.add(Embedding(max_words, embedding_dim, input_length=max_len))
model.add(Conv1D(num_filters, kernel_size, activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [32]:
# Train the model
batch_size = 128
epochs = 15

model.fit(X_train_seq, y_train, batch_size=batch_size, epochs=epochs, validation_data=(X_val_seq, y_val))


Epoch 1/15
104/104 [==============================] - 68s 656ms/step - loss: 0.2754 - accuracy: 0.8979 - val_loss: 1.2535 - val_accuracy: 0.6946
Epoch 2/15
104/104 [==============================] - 68s 652ms/step - loss: 0.2706 - accuracy: 0.8977 - val_loss: 1.2673 - val_accuracy: 0.6883
Epoch 3/15
104/104 [==============================] - 63s 606ms/step - loss: 0.2695 - accuracy: 0.8982 - val_loss: 1.2717 - val_accuracy: 0.6907
Epoch 4/15
104/104 [==============================] - 79s 765ms/step - loss: 0.2704 - accuracy: 0.8975 - val_loss: 1.2820 - val_accuracy: 0.6919
Epoch 5/15
104/104 [==============================] - 77s 741ms/step - loss: 0.2703 - accuracy: 0.8990 - val_loss: 1.2973 - val_accuracy: 0.6910
Epoch 6/15
104/104 [==============================] - 84s 812ms/step - loss: 0.2708 - accuracy: 0.8983 - val_loss: 1.3028 - val_accuracy: 0.6916
Epoch 7/15
104/104 [==============================] - 96s 928ms/step - loss: 0.2700 - accuracy: 0.8978 - val_loss: 1.3070 - val_ac

KeyboardInterrupt: 